In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!ls

ACT_Annotation_L1.txt  Human_Blood plasma.txt query.ipynb
ACT_annotation.ipynb   Human_Blood serum.txt
ACT_marker_genes.ipynb Human_Blood.txt


In [3]:
blood_marker = pd.read_csv('Human_Blood.txt', sep='\t')
plasma_marker = pd.read_csv('Human_Blood plasma.txt', sep='\t')
serum_marker = pd.read_csv('Human_Blood serum.txt', sep='\t')

In [4]:
blood_marker.head()

,Species,Tissue,CellType,Marker,Resource
0,Human,Blood,Activated B cell,"CD5, MIR155HG, TAGLN2","35025971, 28910360, 2474446"
1,Human,Blood,"Activated CD4-positive, alpha-beta T cell","CD4, CCR6, CD14, CD19, CD38, CD3D, CD3G, CD69,...","35381424, 32783921, 35831277, 34529726, 103413..."
2,Human,Blood,"Activated CD4-positive, CD25-positive, CCR4-po...","CD3D, CD3E, CD3G, CD4, FOXP3",30977974
3,Human,Blood,Adult endothelial progenitor cell,"KDR, PROM1, CD34, PTPRC, PECAM1, CD14","27561827, 29391882, 24641802, 29147957, 267700..."
4,Human,Blood,Alpha-beta T cell,"CD3D, CD3E, CD3G, IL7R","34284174, 28777444"


In [5]:
plasma_marker.head()

,Species,Tissue,CellType,Marker,Resource
0,Human,Blood plasma,Endothelial cell,"GP1BA, PECAM1",24511371
1,Human,Blood plasma,Erythrocyte,GYPA,24511371
2,Human,Blood plasma,Erythroid progenitor cell,SLC4A1,35437648
3,Human,Blood plasma,Mature B cell,MS4A1,35437648
4,Human,Blood plasma,Monocyte,CD14,"35437648, 24511371"


In [6]:
plasma_marker['CellType'].unique()

array(['Endothelial cell', 'Erythrocyte', 'Erythroid progenitor cell',
       'Mature B cell', 'Monocyte', 'Neutrophil', 'Plasma cell',
       'Platelet', 'Precursor B cell', 'T cell'], dtype=object)

In [7]:
serum_marker

,Species,Tissue,CellType,Marker,Resource
0,Human,Blood serum,Alternatively activated macrophage,CD163,26886650
1,Human,Blood serum,Macrophage,"CD68, ITGAM, ADGRE1, CSF1R, ITGAX, MRC1","28843510, 28732135, 28323124, 27393389, 275931..."
2,Human,Blood serum,Sertoli cell,TF,2112555


In [8]:
marker_df = pd.concat([blood_marker, plasma_marker])
marker_df.shape

(147, 5)

In [11]:
# unique cell types of the combined data
marker_df['CellType'].unique()

array(['Activated B cell', 'Activated CD4-positive, alpha-beta T cell',
       'Activated CD4-positive, CD25-positive, CCR4-positive, alpha-beta regulatory T cell, human',
       'Adult endothelial progenitor cell', 'Alpha-beta T cell',
       'Alternatively activated macrophage', 'Atypical B cell', 'B cell',
       'Basophil', 'Cardiac muscle cell',
       'CD14-low, CD16-positive monocyte', 'CD14-positive monocyte',
       'CD14-positive, CD16-negative classical monocyte',
       'CD14-positive, CD16-positive monocyte',
       'CD141-positive, CLEC9A-positive dendritic cell',
       'CD16-negative natural killer cell',
       'CD16-negative, CD56-bright natural killer cell, human',
       'CD16-positive myeloid dendritic cell',
       'CD16-positive natural killer cell',
       'CD16-positive, CD56-dim natural killer cell, human',
       'CD1c-positive myeloid dendritic cell', 'CD38-positive B cell',
       'CD4-intermediate, CD8-positive double-positive thymocyte',
       'CD4-posit

In [33]:
def assign_super_type(cell_type):
    cell_type_lower = cell_type.lower()  # Convert to lower case to handle case insensitivity
    if "b cell" in cell_type_lower and "memory b cell" not in cell_type_lower and "naive b cell" not in cell_type_lower:
        return "B"
    if "memory b cell" in cell_type_lower:
        return "B_memory"
    if "naive b cell" in cell_type_lower:
        return "B_naive"
    if "cd4-positive" in cell_type_lower and "cytotoxic t cell" in cell_type_lower:
        return "T_CD4_cytotoxic"
    if "cd4-positive" in cell_type_lower and "memory t cell" in cell_type_lower:
        return "T_CD4_memory"
    if "cd4-positive" in cell_type_lower and "t cell" in cell_type_lower and 'central' not in cell_type_lower and 'effector memory' not in cell_type_lower:
        return "T_CD4"
    if "cd8-positive" in cell_type_lower and "cytotoxic t cell" in cell_type_lower:
        return "T_CD8_cytotoxic"
    if "cd8-positive" in cell_type_lower and "memory t cell" in cell_type_lower:
        return "T_CD8_memory"
    if "cd8-positive" in cell_type_lower and "t cell" in cell_type_lower and 'central' not in cell_type_lower and 'effector memory' not in cell_type_lower:
        return "T_CD8"
    if "monocyte" in cell_type_lower:
        return "Monocyte"
    if "dendritic cell" in cell_type_lower:
        return "DC"
    if "natural killer cell" in cell_type_lower:
        return "NK"
    if "regulatory t cell" in cell_type_lower or "t-regulatory" in cell_type_lower:
        return "T_reg"
    if "central memory t cell" in cell_type_lower:
        return "T_cm"
    if "central memory cd4-positive" in cell_type_lower:
        return "T_CD4_Tcm"
    if "central memory cd8-positive" in cell_type_lower:
        return "T_CD8_Tcm"
    if "effector memory cd4-positive" in cell_type_lower and "t cell" in cell_type_lower:
        return "T_CD4_Tem"
    if "effector memory cd8-positive" in cell_type_lower and "t cell" in cell_type_lower:
        return "T_CD8_Tem"
    if "effector memory" in cell_type_lower and "t cell" in cell_type_lower:
        return "T_em"
    if "erythrocyte" in cell_type_lower:
        return "Erythrocyte"
    if "memory t cell" in cell_type_lower:
        return "T_memory"
    if "proliferating cd4-positive" in cell_type_lower:
        return "T_CD4_proliferating"
    if "proliferating cd8-positive" in cell_type_lower:
        return "T_CD8_proliferating"
    if "helper t cell" in cell_type_lower or "t-helper" in cell_type_lower:
        return "T_helper"
    if "t cell" in cell_type_lower and "cd4-positive" not in cell_type_lower and "cd8-positive" not in cell_type_lower and 'central' not in cell_type_lower and 'effector' not in cell_type_lower:
        return "T"
    if "platelet" in cell_type_lower:
        return "Platelet"
    return ""  # If not matched, return an empty string

# Apply the function to the 'CellType' column
marker_df['super_type'] = marker_df['CellType'].apply(assign_super_type)

marker_df


,Species,Tissue,super_type,CellType,Marker,Resource
0,Human,Blood,B,Activated B cell,"CD5, MIR155HG, TAGLN2","35025971, 28910360, 2474446"
1,Human,Blood,T_CD4,"Activated CD4-positive, alpha-beta T cell","CD4, CCR6, CD14, CD19, CD38, CD3D, CD3G, CD69,...","35381424, 32783921, 35831277, 34529726, 103413..."
2,Human,Blood,T_CD4,"Activated CD4-positive, CD25-positive, CCR4-po...","CD3D, CD3E, CD3G, CD4, FOXP3",30977974
3,Human,Blood,,Adult endothelial progenitor cell,"KDR, PROM1, CD34, PTPRC, PECAM1, CD14","27561827, 29391882, 24641802, 29147957, 267700..."
4,Human,Blood,T,Alpha-beta T cell,"CD3D, CD3E, CD3G, IL7R","34284174, 28777444"
...,...,...,...,...,...,...
5,Human,Blood plasma,,Neutrophil,"CD177, G0S2, LCN2",35437648
6,Human,Blood plasma,,Plasma cell,"CD38, MZB1, SDC1",35437648
7,Human,Blood plasma,Platelet,Platelet,ITGA2B,24511371
8,Human,Blood plasma,B,Precursor B cell,VPREB3,35437648


In [34]:
marker_df.columns

Index(['Species', 'Tissue', 'super_type', 'CellType', 'Marker', 'Resource'], dtype='object')

In [35]:
order = ['Species', 'Tissue', 'super_type', 'CellType', 'Marker', 'Resource']
marker_df = marker_df[order]

In [36]:
marker_df.to_csv('Human_blood_marker_processed.csv', index=False)